In [13]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [14]:
!pip install numpy pandas scikit-learn imbalanced-learn
!pip install usearch

In [22]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

# Load the dataset
df = pd.read_csv('safe_driver_prediction.csv')

# Assuming 'target' is the column with the class labels
# And the rest are features. Adjust the column names as necessary.
X = df.drop('target', axis=1)
y = df['target']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize oversamplers
smote = SMOTE(random_state=0)
adasyn = ADASYN(random_state=0)

# Apply SMOTE and ADASYN
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)


# FastKDE sampling (conceptual, modify according to actual FastKDE implementation)
def fast_kde_sampling(X, y):
    minority_class = X[y == 1]  # Assuming '1' is the minority class
    kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(minority_class)
    new_samples = kde.sample(n_samples=int(len(X) * 0.1), random_state=0)  # Adjust sample size as needed
    new_labels = np.array([1] * len(new_samples))
    return np.vstack([X, new_samples]), np.hstack([y, new_labels])

# Apply FastKDE
X_train_kde, y_train_kde = fast_kde_sampling(X_train, y_train)


# Reset index of y_train after splitting and before using in train_eval_knn
y_train = y_train.reset_index(drop=True)
y_train_smote = y_train_smote.reset_index(drop=True)
y_train_adasyn = y_train_adasyn.reset_index(drop=True)








In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import make_scorer, mean_squared_error
from usearch.index import Index, MetricKind, search



class KNNRegressor:
    def __init__(self, k=5, metric=MetricKind.Cos):
        self.k = k
        self.metric = metric
        self.index = None

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        ndim = X.shape[1]
        self.index = Index(ndim=ndim, metric=self.metric)
        self.index.add(np.arange(len(X)), X)

    def predict(self, X):
        predictions = []

        ff=0
        for x in X:
            ff=ff+1
            print('third: '+str(ff)+' / '+ str(len(X)) +'') if ff % 1000 == 0 else None
            matches = self.index.search(x.reshape(1, -1), count=self.k)
            neighbor_indices = np.array(list(matches.keys))
            if any(index >= len(self.y_train) for index in neighbor_indices):
                print("Out of bounds indices found:", neighbor_indices[neighbor_indices >= len(self.y_train)])
                raise IndexError("Neighbor index out of bounds")
            try:
                avg_target = np.mean(self.y_train[neighbor_indices])  # Using numpy indexing directly
            except IndexError as e:
                print(f"IndexError: {str(e)}")
                continue
            predictions.append(avg_target)
        return np.array(predictions)




    def get_params(self, deep=True):
        # Return parameters as a dictionary
        return {'k': self.k, 'metric': self.metric}

    def set_params(self, **parameters):
        # Set parameters as provided in the dictionary
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self







In [24]:
# Train and evaluate KNN models
def train_eval_knn(X_train, y_train, X_test, y_test, metric=MetricKind.Cos):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define the range for k and initialize tracking variables
    k_values = range(1, 31)
    best_k = None
    lowest_mse = float('inf')

    # Perform k-fold cross-validation manually
    k_fold = KFold(n_splits=5, random_state=0, shuffle=True)
    jj=0
    for k in k_values:
        jj=jj+1
        print('first: '+str(jj)+' / '+ str(len(k_values)) +'')

        mse_accumulated = 0
        temp = k_fold.split(X_train_scaled)
        kk=0

        for train_idx, val_idx in temp:

            kk=kk+1
            print('second: '+str(kk)+' / ')

            # Split the data into k-fold train/validation sets
            X_k_train, X_k_val = X_train_scaled[train_idx], X_train_scaled[val_idx]

            print(len(y_train), "here is ytrain")
            print(train_idx, val_idx, "here is INDEX")

            y_k_train, y_k_val = y_train[train_idx], y_train[val_idx]


            # Train the KNNRegressor with current k
            knn = KNNRegressor(k=k, metric=metric)
            knn.fit(X_k_train, y_k_train)



            # Predict on the validation set
            y_k_pred = knn.predict(X_k_val)



            # Calculate MSE for the current fold and accumulate
            mse_accumulated += mean_squared_error(y_k_val, y_k_pred)

        # Calculate average MSE over all folds
        avg_mse = mse_accumulated / 5

        # Update the best k if the current k results in a lower average MSE
        if avg_mse < lowest_mse:

            lowest_mse = avg_mse
            best_k = k
            print('lowest_mse: '+str(lowest_mse)+'.  best_k: '+str(best_k)+'')


    # Train the model with the best k found from grid search
    best_model = KNNRegressor(k=best_k, metric=metric)
    best_model.fit(X_train_scaled, y_train)

    # Evaluate the model on the test set
    y_pred = best_model.predict(X_test_scaled)
    final_mse = mean_squared_error(y_test, y_pred)

    print(f"Best K: {best_k}")
    print(f"Mean Squared Error: {final_mse}")
    return best_model,  best_k, final_mse


y_train = np.array(y_train.reset_index(drop=True))
y_train_smote = np.array(y_train_smote.reset_index(drop=True))
y_train_adasyn = np.array(y_train_adasyn.reset_index(drop=True))

# Evaluate original, SMOTE, ADASYN, and FastKDE datasets

print("SMOTE Data:")
SMOTE_eval=train_eval_knn(X_train_smote, y_train_smote, X_test, y_test)
print("ADASYN Data:")
ADASYN_eval=train_eval_knn(X_train_adasyn, y_train_adasyn, X_test, y_test)
print("FastKDE Data:")
FastKDE_eval=train_eval_knn(X_train_kde, y_train_kde, X_test, y_test)


SMOTE Data:
first: 1 / 30
second: 1 / 
917572 here is ytrain
[     0      1      2 ... 917568 917569 917571] [     5      6      7 ... 917559 917563 917570] here is INDEX
third: 1000 / 183515
third: 2000 / 183515
third: 3000 / 183515
third: 4000 / 183515
third: 5000 / 183515
third: 6000 / 183515
third: 7000 / 183515
third: 8000 / 183515
third: 9000 / 183515
third: 10000 / 183515
third: 11000 / 183515
third: 12000 / 183515
third: 13000 / 183515
third: 14000 / 183515
third: 15000 / 183515
third: 16000 / 183515
third: 17000 / 183515
third: 18000 / 183515
third: 19000 / 183515
third: 20000 / 183515
third: 21000 / 183515
third: 22000 / 183515
third: 23000 / 183515
third: 24000 / 183515
third: 25000 / 183515
third: 26000 / 183515
third: 27000 / 183515
third: 28000 / 183515
third: 29000 / 183515
third: 30000 / 183515
third: 31000 / 183515
third: 32000 / 183515
third: 33000 / 183515
third: 34000 / 183515
third: 35000 / 183515
third: 36000 / 183515
third: 37000 / 183515
third: 38000 / 183515
th

c:\Users\Shams\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


first: 1 / 30
second: 1 / 
523785 here is ytrain
[     0      1      2 ... 523781 523783 523784] [     5     12     13 ... 523779 523780 523782] here is INDEX
third: 1000 / 104757
third: 2000 / 104757
third: 3000 / 104757
third: 4000 / 104757
third: 5000 / 104757
third: 6000 / 104757
third: 7000 / 104757
third: 8000 / 104757
third: 9000 / 104757
third: 10000 / 104757
third: 11000 / 104757
third: 12000 / 104757
third: 13000 / 104757
third: 14000 / 104757
third: 15000 / 104757
third: 16000 / 104757
third: 17000 / 104757
third: 18000 / 104757
third: 19000 / 104757
third: 20000 / 104757
third: 21000 / 104757
third: 22000 / 104757
third: 23000 / 104757
third: 24000 / 104757
third: 25000 / 104757
third: 26000 / 104757
third: 27000 / 104757
third: 28000 / 104757
third: 29000 / 104757
third: 30000 / 104757
third: 31000 / 104757
third: 32000 / 104757
third: 33000 / 104757
third: 34000 / 104757
third: 35000 / 104757
third: 36000 / 104757
third: 37000 / 104757
third: 38000 / 104757
third: 39000 /

In [18]:
original_eval=train_eval_knn(X_train, y_train, X_test, y_test)

print("Original Data, 1. Best Model, Best K, final_mse: " + str(original_eval))

## DO not run this cell

Original Data, 1. Best Model, Best K, final_mse: (<__main__.KNNRegressor object at 0x000001C33E364800>, 30, 0.03601533339493572)


In [25]:
print("SMOTE Data, 2. Best Model, Best K, final_mse: " + str(SMOTE_eval))

print("ADASYN Data, 3. Best Model, Best K, final_mse: " + str(ADASYN_eval))

print("FastKDE Data, 4. Best Model, Best K, final_mse: " + str(FastKDE_eval))

SMOTE Data, 2. Best Model, Best K, final_mse: (<__main__.KNNRegressor object at 0x000001C373A46C00>, 7, 0.28057551490140675)
ADASYN Data, 3. Best Model, Best K, final_mse: (<__main__.KNNRegressor object at 0x000001C33E4D47A0>, 6, 0.19117368420561381)
FastKDE Data, 4. Best Model, Best K, final_mse: (<__main__.KNNRegressor object at 0x000001C33E2CBD40>, 5, 0.07931184529959763)


Best K: 7
Mean Squared Error: 0.28057551490140675
ADASYN Data:

# https://brukeamare.github.io/AdvancedAML/